In [1040]:
import pandas as pd
import numpy as np

In [1041]:

#SAME CODE AS WEEK 2 !



filename = "../.gitignore/data_a (Florent).csv"
df = pd.read_csv(filename, skipinitialspace=True) #Open the csv file with no space before and after the values

#Part 1 : cleaning the data

#A. Drop the duplicates

nb_of_duplicates = len(df)-len(df.drop_duplicates()) #How many duplicates do I have ?

if nb_of_duplicates != 0 :
    df.drop_duplicates(inplace=True)

#B. Empty cells :

df.isnull().sum() #How many empty cells are there for each column

cleaned_df = df.dropna(subset=['Price'])#Delete the 3992 empty price
cleaned_df = cleaned_df.dropna(subset=['Number of rooms'])#Delete the 1918 empty Number of rooms
cleaned_df = cleaned_df.dropna(subset=['Living Area'])#Delete the 11677 empty Living Area (60605 left)

cleaned_df["Garden"].fillna(False, inplace = True) #Replace NaN value by False in the Garden Column
cleaned_df["Furnished"].fillna(False, inplace = True) #Replace NaN value by False in the Furnished Column
cleaned_df["Fully equipped kitchen"].fillna(False, inplace = True) #Replace NaN value by False in the Furnished Column

del cleaned_df['Area of the terrace'] #Delete the column Area of the terrace
del cleaned_df['Area of the garden'] #Delete the column Area of the garden
#del cleaned_df['State of the building'] #Delete the column State of the building
del cleaned_df['Surface area of the plot of land'] #Delete the column Surface area of the plot of land
del cleaned_df['Surface of the land'] #Delete the column Surface area of land
del cleaned_df['Number of facades'] #Delete the column Number of facades

nb_null_values = cleaned_df.isnull().sum()
#C. No blank spaces 
# Already done in the opening of the csv file

#D. Data in wrong format

cleaned_df_formatted = cleaned_df.convert_dtypes() #convert all the dtypes in the best types for them
cleaned_df_formatted['Price'] = cleaned_df_formatted['Price'].astype(float)
cleaned_df_formatted['Living Area'] = cleaned_df_formatted['Living Area'].astype(float)
cleaned_df_formatted['Number of rooms'] = cleaned_df_formatted['Number of rooms'].astype(float)

#E. Wrong data

index_sell_to_del = cleaned_df_formatted[(cleaned_df_formatted['To sell'] == False)].index # taking the index of the row who are False to delete them on the next line
df_sell = cleaned_df_formatted.drop(index_sell_to_del) #Delete the rows collect the line above and store the new DF in df_sell
del df_sell['To rent'] #Delete the column "To rent" because it is useless now

index_rent_to_del = cleaned_df_formatted[(cleaned_df_formatted['To rent'] == False)].index # taking the index of the row who are False to delete them on the next line
df_rent = cleaned_df_formatted.drop(index_rent_to_del) #Delete the rows collect the line above and store the new DF in df_rent
del df_rent['To sell'] #Delete the column "To sell" because it is useless now

#Delete the extreme prices and extreme living area from df_rent

df_rent.drop(df_rent[df_rent['Price'] >= 4000].index, inplace = True)
df_rent.drop(df_rent[df_rent['Living Area'] > 400].index, inplace = True)

#Creating a Dataframe for renting appartment and Rez-de-chaussee

#Taking only the appartment and Rez-de-chaussée to rent 
df_rent_appart = df_rent[(df_rent['type'] == "Appartement") | (df_rent['type'] == "Rez-de-chaussée")].copy()

#Calculating the price per squared m for the appartment and Rez-de-chaussée to rent
df_rent_appart.loc[ : ,'Price by squared m'] = df_rent_appart.loc[ : ,'Price'] / df_rent_appart.loc[ : ,'Living Area']

#Calculating the price per bedroom m for the appartment and Rez-de-chaussée to rent
#df_rent_appart['Price by bedroom'] = df_rent_appart['Price'] / df_rent_appart['Number of rooms']


#Creating Datafram for selling appartement and Rez-de-chaussée

#Taking only the appartment and Rez-de-chausséet to sell 
df_sell_appart = df_sell[(df_sell['type'] == "Appartement") | (df_sell['type'] == "Rez-de-chaussée")]
#df_sell_appart.drop(df_sell_appart[(df_sell_appart['Price'] < 50000)].index, inplace=True) #Deleting the rows where the price are under 50 000 because some renting appart were true to sell so...

#Part 2 : Data Analysis

#How many rows and columns ?

def how_many_columns_rows_df(df):
    vnames = [name for name in globals() if globals()[name] is df]#Getting the name of the df
    print(f"There are {df.shape[0]} rows in our database named {vnames[0]}") #How many rows ?
    print(f"There are {df.shape[1]} columns in our database named {vnames[0]}") #How many columns ?
    return None

#how_many_columns_rows_df(df_rent_appart)

In [1042]:
#Preprocessing Data

#How many rows and columns in my DF ?

how_many_columns_rows_df(df_rent_appart)
how_many_columns_rows_df(df_sell_appart)

#Calculating the correlation of my dataframe(s)

corr = df_rent_appart.corr(numeric_only=True).abs() #Calculate the correlation between all the variables in the DF and take the absolute value of them

print(corr['Price'])
#Seeing the results of the correlation with the price I decide to delete some columns
#who are not meaningful in my case

print(corr['Price'][corr['Price']<0.36]) #The columns that I'll delete from my df


del df_rent_appart['Unnamed: 0']
del df_rent_appart['Fully equipped kitchen']
#del df_rent_appart['Open fire']
#del df_rent_appart['Garden']
del df_rent_appart['Swimming pool']
del df_rent_appart['type']
del df_rent_appart['To rent']
del df_rent_appart['Price by squared m']

df_rent_appart.head(1)




There are 8676 rows in our database named df_rent_appart
There are 15 columns in our database named df_rent_appart
There are 15543 rows in our database named df_sell_appart
There are 14 columns in our database named df_sell_appart
Unnamed: 0                0.119638
To rent                        NaN
Price                     1.000000
Number of rooms           0.393882
Living Area               0.606735
Fully equipped kitchen    0.069420
Furnished                 0.259539
Open fire                 0.092814
Terrace                   0.189533
Garden                    0.038577
Swimming pool             0.061990
zipcode                   0.368450
Price by squared m        0.544780
Name: Price, dtype: float64
Unnamed: 0                0.119638
Fully equipped kitchen    0.069420
Furnished                 0.259539
Open fire                 0.092814
Terrace                   0.189533
Garden                    0.038577
Swimming pool             0.061990
Name: Price, dtype: float64


,Price,Number of rooms,Living Area,Furnished,Open fire,Terrace,Garden,State of the building,zipcode
1,790.0,2.0,140.0,False,False,False,False,Bon,3200


In [1043]:
#Creating the column "Province"

df_rent_appart[(df_rent_appart["zipcode"] >= 1000) & (df_rent_appart["zipcode"] <= 1299)]

Province = []
for zipcode in df_rent_appart['zipcode']:
    if (zipcode >= 1000) & (zipcode <= 1299): Province.append('Brussels')
    elif (zipcode >= 1300) & (zipcode <= 1499): Province.append('Brabant wallon')
    elif (zipcode >= 2000) & (zipcode <= 2999): Province.append('Anvers')
    elif (zipcode >= 3500) & (zipcode <= 3999): Province.append('Limbourg')
    elif (zipcode >= 4000) & (zipcode <= 4999): Province.append('Liege')
    elif (zipcode >= 5000) & (zipcode <= 5680): Province.append('Namur')
    elif (zipcode >= 6600) & (zipcode <= 6999): Province.append('Luxembourg')
    elif (zipcode >= 8000) & (zipcode <= 8999): Province.append('Flandre occiendentale')
    elif (zipcode >= 9000) & (zipcode <= 9999): Province.append('Flandre orientale')
    elif (zipcode >= 1500) & (zipcode <= 1999) or (zipcode >= 3000) & (zipcode <= 3499): Province.append('Brabant flamand')
    elif (zipcode >= 6000) & (zipcode <= 6599) or (zipcode >= 7000) & (zipcode <= 7999): Province.append('Hainaut')

df_rent_appart['Province'] = Province
del df_rent_appart['zipcode'] #Not needed anymore


In [1044]:
df_rent_appart = pd.get_dummies(data = df_rent_appart, columns = ['Province', 'State of the building'])
df_rent_appart.head(10)
how_many_columns_rows_df(df_rent_appart)

There are 8676 rows in our database named df_rent_appart
There are 24 columns in our database named df_rent_appart


In [1045]:
#Modelization part for renting appart in Belgium with LinearRegression

#Defining the features and the target

print(df_rent_appart.columns)

X = np.array(df_rent_appart.drop(columns = 'Price')) #features = Number of rooms, Living area and zipcode
y = np.array(df_rent_appart['Price']) #target = Renting price

#print(f"That's a sample line for the X_features : {X}")

#Splitting the data between the training part and the testing part
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size = 0.2, random_state = 42 )


#Calling the algorithm we want to apply on our model and applying it
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)

#Asking for the score of our model for its training part
training_score = regressor.score(X_train, y_train)
print(f"The score of the model for training part : {training_score}")


#Making a prediction with our model on the testing data part
regressor.fit(X_test, y_test)
regressor.predict(X_test)

#Asking the score of our model for its testing part
test_score = regressor.score(X_test, y_test)
print(f"The score of the model for testing part : {test_score}")

Index(['Price', 'Number of rooms', 'Living Area', 'Furnished', 'Open fire',
       'Terrace', 'Garden', 'Province_Anvers', 'Province_Brabant flamand',
       'Province_Brabant wallon', 'Province_Brussels',
       'Province_Flandre occiendentale', 'Province_Flandre orientale',
       'Province_Hainaut', 'Province_Liege', 'Province_Limbourg',
       'Province_Luxembourg', 'Province_Namur',
       'State of the building_\nBon\n',
       'State of the building_\nExcellentétat\n',
       'State of the building_\nFraîchementrénové\n',
       'State of the building_\nÀrafraîchir\n',
       'State of the building_\nÀrestaurer\n',
       'State of the building_\nÀrénover\n'],
      dtype='object')
The score of the model for training part : 0.6276519353202796
The score of the model for testing part : 0.6175876547912615


In [1046]:
#Modelization part for renting appart in Belgium with Lasso

#Defining the features and the target

X = np.array(df_rent_appart.drop(columns = 'Price')) #features = Number of rooms, Living area and zipcode
y = np.array(df_rent_appart['Price']) #target = Renting price

#print(f"That's a sample line for the X_features : {X}")

#Splitting the data between the training part and the testing part
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size = 0.2, random_state = 42 )


#Calling the algorithm we want to apply on our model and applying it
from sklearn.linear_model import Lasso

regressor = Lasso(alpha=0.01)
regressor.fit(X_train, y_train)

#Asking for the score of our model for its training part
training_score = regressor.score(X_train, y_train)
print(f"The score of the model for training part : {training_score}")


#Making a prediction with our model on the testing data part
regressor.fit(X_test, y_test)
regressor.predict(X_test)

#Asking the score of our model for its testing part
test_score = regressor.score(X_test, y_test)
print(f"The score of the model for testing part : {test_score}")

The score of the model for training part : 0.627648936770111
The score of the model for testing part : 0.617585870578557


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.997e+06, tolerance: 4.262e+04
  model = cd_fast.enet_coordinate_descent(


In [1047]:
#Modelization part for renting appart in Belgium with Ridge Regression

#Defining the features and the target

X = np.array(df_rent_appart.drop(columns = 'Price')) #features = Number of rooms, Living area and zipcode
y = np.array(df_rent_appart['Price']) #target = Renting price

#print(f"That's a sample line for the X_features : {X}")

#Splitting the data between the training part and the testing part
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size = 0.2, random_state = 42 )


#Calling the algorithm we want to apply on our model and applying it
from sklearn.linear_model import Ridge

regressor = Ridge(alpha=0.01)
regressor.fit(X_train, y_train)

#Asking for the score of our model for its training part
training_score = regressor.score(X_train, y_train)
print(f"The score of the model for training part : {training_score}")


#Making a prediction with our model on the testing data part
regressor.fit(X_test, y_test)
regressor.predict(X_test)

#Asking the score of our model for its testing part
test_score = regressor.score(X_test, y_test)
print(f"The score of the model for testing part : {test_score}")

The score of the model for training part : 0.6276519341837976
The score of the model for testing part : 0.6175876001001257


In [1048]:
#Modelization part for renting appart in Belgium with Elastic Net

#Defining the features and the target

X = np.array(df_rent_appart.drop(columns = 'Price')) #features = Number of rooms, Living area and zipcode
y = np.array(df_rent_appart['Price']) #target = Renting price

#print(f"That's a sample line for the X_features : {X}")

#Splitting the data between the training part and the testing part
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size = 0.2, random_state = 42 )


#Calling the algorithm we want to apply on our model and applying it
from sklearn.linear_model import ElasticNet

regressor = ElasticNet(alpha=0.01)
regressor.fit(X_train, y_train)

#Asking for the score of our model for its training part
training_score = regressor.score(X_train, y_train)
print(f"The score of the model for training part : {training_score}")


#Making a prediction with our model on the testing data part
regressor.fit(X_test, y_test)
regressor.predict(X_test)

#Asking the score of our model for its testing part
test_score = regressor.score(X_test, y_test)
print(f"The score of the model for testing part : {test_score}")

The score of the model for training part : 0.6271718504470398
The score of the model for testing part : 0.6167245073721328


In [1049]:
#FOR RENT_DF














In [1050]:
#Preprocessing Data

#How many rows and columns in my DF ?

how_many_columns_rows_df(df_rent)

#Calculating the correlation of my dataframe(s)

corr = df_rent.corr(numeric_only=True).abs() #Calculate the correlation between all the variables in the DF and take the absolute value of them

print(corr['Price'])
#Seeing the results of the correlation with the price I decide to delete some columns
#who are not meaningful in my case

del df_rent['Unnamed: 0']
del df_rent['Fully equipped kitchen']
#del df_rent['Open fire']
#del df_rent['Garden']
del df_rent['Swimming pool']
#del df_rent['type']
del df_rent['To rent']


There are 12773 rows in our database named df_rent
There are 14 columns in our database named df_rent
Unnamed: 0                0.074634
To rent                        NaN
Price                     1.000000
Number of rooms           0.483606
Living Area               0.608514
Fully equipped kitchen    0.072724
Furnished                 0.170442
Open fire                 0.166774
Terrace                   0.205782
Garden                    0.152727
Swimming pool             0.069647
zipcode                   0.377538
Name: Price, dtype: float64


In [1051]:
#Creating the column "Province"

Province = []

for zipcode in df_rent['zipcode']:
    if (zipcode >= 1000) & (zipcode <= 1299): Province.append('Brussels')
    elif (zipcode >= 1300) & (zipcode <= 1499): Province.append('Brabant wallon')
    elif (zipcode >= 2000) & (zipcode <= 2999): Province.append('Anvers')
    elif (zipcode >= 3500) & (zipcode <= 3999): Province.append('Limbourg')
    elif (zipcode >= 4000) & (zipcode <= 4999): Province.append('Liege')
    elif (zipcode >= 5000) & (zipcode <= 5680): Province.append('Namur')
    elif (zipcode >= 6600) & (zipcode <= 6999): Province.append('Luxembourg')
    elif (zipcode >= 8000) & (zipcode <= 8999): Province.append('Flandre occiendentale')
    elif (zipcode >= 9000) & (zipcode <= 9999): Province.append('Flandre orientale')
    elif (zipcode >= 1500) & (zipcode <= 1999) or (zipcode >= 3000) & (zipcode <= 3499): Province.append('Brabant flamand')
    elif (zipcode >= 6000) & (zipcode <= 6599) or (zipcode >= 7000) & (zipcode <= 7999): Province.append('Hainaut')

df_rent['Province'] = Province
del df_rent['zipcode'] #Not needed anymore

In [1052]:
df_rent.isnull().sum()
df_rent.columns

Index(['Price', 'Number of rooms', 'Living Area', 'Furnished', 'Open fire',
       'Terrace', 'Garden', 'State of the building', 'type', 'Province'],
      dtype='object')

In [1053]:
df_rent.head(2)

#df_rent.groupby('type').count()
df_rent = pd.get_dummies(data = df_rent, columns = ['Province','type','State of the building'])

In [1054]:
#Modelization part for renting appart in Belgium with LinearRegression

#Defining the features and the target

X = np.array(df_rent.drop(columns = 'Price')) #features = Number of rooms, Living area and zipcode
y = np.array(df_rent['Price']) #target = Renting price

#print(f"That's a sample line for the X_features : {X}")

#Splitting the data between the training part and the testing part
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size = 0.2, random_state = 42 )


#Calling the algorithm we want to apply on our model and applying it
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)

#Asking for the score of our model for its training part
training_score = regressor.score(X_train, y_train)
print(f"The score of the model for training part : {training_score}")


#Making a prediction with our model on the testing data part
regressor.predict(X_test)

#Asking the score of our model for its testing part
test_score = regressor.score(X_test, y_test)
print(f"The score of the model for testing part : {test_score}")

The score of the model for training part : 0.6481475555173728
The score of the model for testing part : 0.6184343118938049


In [1055]:
#Modelization part for renting appart in Belgium GradientBoostingRegressor

#Defining the features and the target

X = np.array(df_rent_appart.drop(columns = 'Price')) #features = 
y = np.array(df_rent_appart['Price']) #target = Renting price

#print(f"That's a sample line for the X_features : {X}")

#Splitting the data between the training part and the testing part
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size = 0.2, random_state = 42 )


#Calling the algorithm we want to apply on our model and applying it

from sklearn.ensemble import GradientBoostingRegressor

regressor = GradientBoostingRegressor()
regressor.fit(X_train, y_train)

#Asking for the score of our model for its training part
training_score = regressor.score(X_train, y_train)
print(f"The score of the model for training part : {training_score}")


#Making a prediction with our model on the testing data part

regressor.predict(X_test)

#Asking the score of our model for its testing part
test_score = regressor.score(X_test, y_test)
print(f"The score of the model for testing part : {test_score}")

#Define one new instance
#A random sample
use_samples = [2.0, 90, False, False, False, False, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
samples_to_predict = []

# Convert into Numpy array
Xnew = np.array(use_samples).reshape((1,-1))

# make a prediction
ynew = regressor.predict(Xnew)
print(f"Renting Price Predicted = {ynew[0].round(2)} €")

The score of the model for training part : 0.7458246962614868
The score of the model for testing part : 0.7102087581036436
Renting Price Predicted = 1218.86 €


In [1056]:
#Standardisation of the data
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

#Normalisation of the data
#from sklearn.preprocessing import Normalizer
#sc = Normalizer()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)